In [1]:
'''
Programmer : Agus Nursikuwagus
Create :  January, 3, 2023
This program is an implementation of image captioning geological rocks images.
Some issues related to other model and tested with baseline model.
We called the program is SemAtt Captioning
'''

import numpy as np
from numpy import array

import matplotlib.pyplot as plt
#%matplotlib inline


import string
import os
import glob
import tensorflow as tf

from time import time

from keras import Input, layers
from keras import optimizers

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Dense, Activation, Flatten, Reshape, Dropout
from tensorflow.keras.layers import add
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
token_path = "/content/drive/MyDrive/dataset_captioning/Flickr8K_Text/Flickr8k.token.txt"
train_images_path = '/content/drive/MyDrive/dataset_captioning/Flickr8K_Text/Flickr_8k.trainImages.txt'
test_images_path = '/content/drive/MyDrive/dataset_captioning/Flickr8K_Text/Flickr_8k.testImages.txt'
images_path = '/content/drive/MyDrive/dataset_captioning/Flicker8k_Dataset/'
test_path ='/content/drive/MyDrive/dataset_captioning/test_image/'
glove_path = '/content/drive/MyDrive/Colab Notebooks/embedding/'

doc = open(token_path,'r').read()
print(doc[:700])

1.jpg#0 Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, batulumpur karbonatan, masif, retak-retak, sebagian hancur dan mulai lapuk
1.jpg#1 Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, masif, retak-retak, sebagian hancur sehingga mulai lapuk dan batulumpur karbonatan
1.jpg#2 Singkapan batuan sedimen klastik dan batulumpur karbonatan 
1.jpg#3 batulumpur karbonatan dan Singkapan batuan sedimen klastik
1.jpg#4 Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas dan batulumpur karbonatan
2.jpg#0 Pecahan koral
2.jpg#1 Pecahan koral
2.jpg#2 Pecahan koral
2.jpg#3 Pecahan koral
2.jpg#4 Pecahan koral
3.jpg#0 Singkapan Ba


In [5]:
descriptions = dict()
for line in doc.split('\n'):
        tokens = line.split()
        if len(line) > 2:
            image_id = tokens[0].split('.')[0]
            image_desc = ' '.join(tokens[1:])
            if image_id not in descriptions:
                descriptions[image_id] = list()
            descriptions[image_id].append(image_desc)

In [6]:
table = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [w.translate(table) for w in desc]
        desc_list[i] =  ' '.join(desc)

In [7]:
vocabulary = set()
for key in descriptions.keys():
        [vocabulary.update(d.split()) for d in descriptions[key]]
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 397


In [8]:
lines = list()
for key, desc_list in descriptions.items():
    for desc in desc_list:
        lines.append(key + ' ' + desc)

new_descriptions = '\n'.join(lines)

In [9]:
doc = open(train_images_path,'r').read()
dataset = list()
for line in doc.split('\n'):
    if len(line) > 1:
      identifier = line.split('.')[0]
      dataset.append(identifier)

train = set(dataset)

In [10]:
img = glob.glob(images_path + '*.jpg')
train_images = set(open(train_images_path, 'r').read().strip().split('\n'))
train_img = []
for i in img:
    if i[len(images_path):] in train_images:
        train_img.append(i)

#print(train_img)

print(test_path)
imgs = glob.glob(test_path + '*.jpg')
test_images = set(open(test_images_path, 'r').read().strip().split('\n'))
print(test_images)
test_img = []
for j in imgs:
    if j[len(test_path):] in test_images:
        test_img.append(j)

#print(test_img)

/content/drive/MyDrive/dataset_captioning/test_image/
{'403.jpg', '424.jpg', '556.jpg', '507.jpg', '779.jpg', '568.jpg', '758.jpg', '825.jpg', '335.jpg', '443.jpg', '723.jpg', '720.jpg', '217.jpg', '206.jpg', '241.jpg', '248.jpg', '316.jpg', '505.jpg', '642.jpg', '43.jpg', '110.jpg', '188.jpg', '775.jpg', '810.jpg', '305.jpg', '693.jpg', '699.jpg', '615.jpg', '37.jpg', '609.jpg', '717.jpg', '571.jpg', '603.jpg', '831.jpg', '434.jpg', '665.jpg', '383.jpg', '267.jpg', '117.jpg', '672.jpg', '182.jpg', '196.jpg', '176.jpg', '532.jpg', '11.jpg', '7.jpg', '19.jpg', '394.jpg', '529.jpg', '339.jpg', '539.jpg', '821.jpg', '33.jpg', '650.jpg', '840.jpg', '420.jpg', '518.jpg', '254.jpg', '590.jpg', '812.jpg', '490.jpg', '451.jpg', '156.jpg', '787.jpg', '88.jpg', '848.jpg', '356.jpg', '272.jpg', '413.jpg', '653.jpg', '581.jpg', '679.jpg', '99.jpg', '129.jpg'}


In [11]:
train_descriptions = dict()
for line in new_descriptions.split('\n'):
    tokens = line.split()
    image_id, image_desc = tokens[0], tokens[1:]
    if image_id in train:
        if image_id not in train_descriptions:
            train_descriptions[image_id] = list()
        desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
        train_descriptions[image_id].append(desc)

In [12]:
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)

In [13]:
word_count_threshold = 0
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('Vocabulary = %d' % (len(vocab)))

Vocabulary = 379


In [14]:
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1

In [15]:
all_desc = list()
for key in train_descriptions.keys():
    [all_desc.append(d) for d in train_descriptions[key]]
lines = all_desc
max_length = max(len(d.split()) for d in lines)

print('Description Length: %d' % max_length)

Description Length: 22


In [16]:
embeddings_index = {}
f = open(os.path.join(glove_path, 'geo_glove.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

In [17]:
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [18]:
model = tf.keras.applications.xception.Xception()



91884032/91884032 [==============================] - 1s 0us/step


In [19]:
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])


In [21]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
print(history.history['loss'])
print(history.history['accuracy'])

model.save('./input/xception_lstm.h5')

In [23]:
from nltk.translate.bleu_score import corpus_bleu
from keras.models import load_model

model = load_model('./input/xception_lstm.h5')

def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break

    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)


    return final

OSError: ignored

In [ ]:
import pandas as pd

def beam_search_predictions(image, beam_index = 3):
    start = [wordtoix["startseq"]]
    start_word = [[start, 0.0]]
    i = 0
    while len(start_word[0][0]) < max_length:
        temp = []
        for s in start_word:
            par_caps = sequence.pad_sequences([s[0]], maxlen=max_length, padding='post')
            #print(par_caps)
            #print('\n')

            preds = model.predict([image,par_caps], verbose=0)
            #print(preds)
            #print('\n')

            word_preds = np.argsort(preds[0])[-beam_index:]
            # Getting the top <beam_index>(n) predictions and creating a
            # new list so as to put them via the model again
            for w in word_preds:

                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
                prob += preds[0][w]
                temp.append([next_cap, prob])
                df = pd.DataFrame(temp, columns = ['Sequence Word','Probabilty'])
                #print(df.head(10))
                #print('\n')


        start_word = temp
        # Sorting according to the probabilities
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        # Getting the top words
        start_word = start_word[-beam_index:]
        df_word = pd.DataFrame(start_word)
        #print('\n')


    start_word = start_word[-1][0]
    intermediate_caption = [ixtoword[i] for i in start_word]
    final_caption = []

    for i in intermediate_caption:
        if i != 'endseq':
            final_caption.append(i)
        else:
            break

    final_caption = ' '.join(final_caption[1:])
    #print(final_caption)

    return final_caption

In [ ]:
import tensorflow as tf
import nltk
nltk.download('punkt')

from nltk import word_tokenize

def tokeniser(text_generation):
    token = []
    #i = 0
    for in_txt in text_generation:
        refer = word_tokenize(in_txt)
        token.append(refer)
        #print(token[i])
        #i =+ 1


    list_token = token
    return list_token

#print(tokeniser(descriptions['1']))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\agus2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk.translate.bleu_score import sentence_bleu


for img in list(encoding_train.keys()):
    print(img)
    picture = plt.imread(images_path + img)
    print('nama file = ' + img)
    image = encoding_train[img].reshape(1,4096)
    plt.imshow(picture)
    plt.show()

    text = img.split('.')[0]
    print(descriptions[text])
    references = tokeniser(descriptions[text])
    #print('\n')


    print("Greedy:",greedySearch(image))

    print("Beam Search, K = 3:",beam_search_predictions(image, beam_index = 3))

    candidates = word_tokenize(beam_search_predictions(image, beam_index = 3))
    #print("\n")

    #print("Beam Search, K = 5:",beam_search_predictions(image, beam_index = 5))
    #print("\n")

    print('BLEU-1: %f' % sentence_bleu(references, candidates, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % sentence_bleu(references, candidates, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % sentence_bleu(references, candidates, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % sentence_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25)))
    print("\n")

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
import seaborn as sns

pic = '1.jpg'
image = encoding_test[pic].reshape((1,4096))
x=plt.imread(test_path + pic)
plt.imshow(x)
plt.show()

print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print("Greedy Search Predictions :",greedySearch(image))
print('\n')

print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print("Beam Search Predictions, K = 3:",beam_search_predictions(image, beam_index = 3))
print('\n')

print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print("Beam Search Predictions, K = 5:",beam_search_predictions(image, beam_index = 5))
print('\n')


references = tokeniser(descriptions['1'])
print(references)

candidates = word_tokenize(beam_search_predictions(image, beam_index = 3))

print(references)
#print(candidates)

print('\n')

print('BLEU-1: %f' % sentence_bleu(references, candidates, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % sentence_bleu(references, candidates, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % sentence_bleu(references, candidates, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %f' % sentence_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25)))




In [ ]:
'''
import csv
with open('./input/incept_lstm_blue.csv','w', newline='') as f:
  for ndx in captions:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(ndx)


import csv
with open('./input/xcept_lstm_rouge.csv','w', newline='') as f:
  for ndx in rougelist:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(ndx)

'''

import csv
with open('./input/xcept_lstm_meteor.csv','w', newline='') as f:
  for ndx in meteorlist:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(ndx)